# 🔀 Week 09-10 · Notebook 07 · Fine-tuning vs. RAG Decision Matrix

Quantify when manufacturing teams should invest in fine-tuning versus improving retrieval-augmented generation pipelines.
code
python
markdown
## 🎯 Learning Objectives
- Capture business, compliance, and technical drivers for adaptation choices.
- Build a cost/benefit calculator referencing plant data.
- Compare pilot case studies linking back to Weeks 05-08 RAG builds.
- Produce a stakeholder-ready recommendation memo.
markdown
#VSC-990cdbd5
markdown
## 🧩 Scenario
A manufacturing PM must decide how to upgrade a warranty-claims assistant. Options: enhance the existing RAG pipeline (Week 08) or fine-tune a domain model (this module).
code
#VSC-567c5afd
python
import pandas as pd
import numpy as np
from IPython.display import Markdown
markdown
#VSC-48f7d702
markdown
## 🧮 Cost / Benefit Calculator
Estimate monthly TCO and performance impact for each strategy.
code
#VSC-1166af88
python
inputs = {
levers = pd.DataFrame([
levers
markdown
#VSC-af3f9f09
markdown
### 📊 Weighted Decision Score
Adjust weights to reflect stakeholder priorities (Ops, Compliance, IT).
code
#VSC-34b2e5b2
python
weights = {
def score_option(row):
    return sum(row[field] * weight for field, weight in weights.items())

levers['weighted_score'] = levers.apply(score_option, axis=1)
levers.sort_values('weighted_score', ascending=False)
markdown
#VSC-0d862747
markdown
## 🧾 Decision Matrix Template
| Dimension | RAG Upgrade | Fine-tune | Hybrid | Notes |
markdown
#VSC-35b953de
markdown
## 🧪 Case Study Snapshots
1. **Maintenance Co-pilot (Week 08)**: RAG upgrade increased top-3 precision by 18% with minimal cost.
2. **EHS Incident Generator**: Fine-tuning reduced hallucinated procedures but required 4 FTE months for data labeling.
3. **Supplier Quality**: Hybrid approach offered 15% latency improvements with safe fallback to retrieval.
markdown
#VSC-c5366cbb
markdown
## 💬 Stakeholder Alignment Worksheet
| Role | Questions to Ask | Decision Rights |
 | Approve latency SLA |
 | Approve governance plan |
 | Approve infra changes |
 | Validate accuracy targets |
markdown
#VSC-3667f5ec
markdown
## 🧑‍💼 Generate Recommendation Memo
Summarize insights for leadership.
markdown
#VSC-6f5fb79d
markdown
memo = f




Markdown(memo)
markdown
#VSC-92504e06
markdown
## 🧪 Lab Assignment
1. Populate the calculator with your plant's actual metrics.
2. Run sensitivity analysis: change compliance weight from 0.3 → 0.6, observe decision shifts.
3. Draft a hybrid roadmap (fine-tune adapter + RAG) and estimate effort.
4. Present a 5-slide deck to stakeholders summarizing findings.
markdown
#VSC-a51a2c50
markdown
## ✅ Checklist
- [ ] Inputs gathered from Ops, Compliance, IT
- [ ] Calculator populated with realistic costs
- [ ] Decision memo drafted
- [ ] Stakeholder Q&A logged
markdown
#VSC-fb8b775f
markdown
## 📚 References
- Deloitte Manufacturing AI: *Adaptation Playbooks* (2025)
- HuggingFace: *RAG vs. Fine-tuning Trade-offs*
- Week 05-08 Notebook Links